<a href="https://colab.research.google.com/github/pko89403/DeepLearningSelfStudy/blob/master/sequence_models_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시퀀스 모델과 LSTM 네트워크
이전에 많은 모델을 보았지만, 네트워크에서 전적으로 유지되는 스테이트는 없다. 시퀀스 모델은  NLP에서 메인이다. 여기서 말하는 시퀀스 모델은 인풋 사이의 시간에 어느정도 의존하는 모델이다. 전통적인 시퀀스 모델은 히든 마코프 모델. 
RNN 모델은 어떤 상태를 유지하는 모델이다. 예를 들어, 아웃풋이 다음 입력의 한 파트로 사용될 수도 있다. 따라서 정보가 시퀀스에 대해 네트워크를 따라 전파될 수 있다. LSTM의 경우, 시퀀스의 각 요소들이 히든 스테이트로 할당 될 수 있다. 히든 스테이트를 예측하는데 사용할 수 있다.

## 파이토치 에서의 LSTM
파이토치에서 LSTM은 모든 입력들을 3D 텐서로 예상한다. 이러한 텐서들이 가지는 축들의 의미는 매우 중요하다. 
- 첫번째 축은 시퀀스, 데이터가 가지는 타임스탬프의 길이 자체다.
- 두번째 인덱스는 mini-batch size, 데이터 배치 사이즈
- 그리고 세번째 인덱스는 한 타임스탬프가 가지는 입력 데이터의 차원

> The cow jumped 는 (1,1,3)이 된다.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [0]:
lstm = nn.LSTM(3, 3) # Input dim is 3, Output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)] # make a sequence of length 5

In [7]:
# 히든 스테이트를 초기화
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
  # 한 번에 한 요소 씩 시퀀스를 단계 별로 진행한다
  # 각 단계 후에, 히든은 히든 스테이트를 포함한다.
  out, hidden = lstm(i.view(1, 1, -1), hidden)
  print('out\t' , out, '\thidden\t', hidden)

  # 대안으로, 전 시퀀스를 한번에 다 할 수 있다.
  # LSTM에서 반환된 첫번째 값은 모든 히든 스테이트 결과들이다
  # 시퀀스다. 두번째는 가장 최근의 히든 스테이트다
  # ( out의 마지막 조각과 hidden을 비교하면 같다. )
  # "out"은 시퀀스 내의 모든 히든 스테이트에 접근하게 한다.
  # "hidden"은 시퀀스와 백-프로파게이션을 계속하는 것을 허용한다
  # 마지막 time에 argument로 lstm에 전달하면
  # 추가 2차원을 더한다

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3)) 
out, hidden = lstm(inputs, hidden)
print('Out - ', out)
print('Hidden - ', hidden)

out	 tensor([[[0.2179, 0.0546, 0.2426]]], grad_fn=<StackBackward>) 	hidden	 (tensor([[[0.2179, 0.0546, 0.2426]]], grad_fn=<StackBackward>), tensor([[[0.3195, 0.1672, 0.9442]]], grad_fn=<StackBackward>))
out	 tensor([[[0.0585, 0.0731, 0.2434]]], grad_fn=<StackBackward>) 	hidden	 (tensor([[[0.0585, 0.0731, 0.2434]]], grad_fn=<StackBackward>), tensor([[[0.0950, 0.1684, 0.6035]]], grad_fn=<StackBackward>))
out	 tensor([[[ 0.1322, -0.0984,  0.0183]]], grad_fn=<StackBackward>) 	hidden	 (tensor([[[ 0.1322, -0.0984,  0.0183]]], grad_fn=<StackBackward>), tensor([[[ 0.2221, -0.4208,  0.0376]]], grad_fn=<StackBackward>))
out	 tensor([[[ 0.1115, -0.1652,  0.0573]]], grad_fn=<StackBackward>) 	hidden	 (tensor([[[ 0.1115, -0.1652,  0.0573]]], grad_fn=<StackBackward>), tensor([[[ 0.1788, -0.4906,  0.1991]]], grad_fn=<StackBackward>))
out	 tensor([[[0.0456, 0.1216, 0.0614]]], grad_fn=<StackBackward>) 	hidden	 (tensor([[[0.0456, 0.1216, 0.0614]]], grad_fn=<StackBackward>), tensor([[[0.0836, 0.2870, 0.11

## 예제 : Part-of-Speech 태깅을 위한 LSTM
LSTM을 speech tag의 부분을 얻기 위해 사용한다.   
문장을 입력해서 vocab의 word들로 이루어졌다. 예측은 word를 예측하고
구조적으로 보았을 때 모델의 아웃풋은 태그들로 이루어진 시퀀스다.    
예측을 하기위해 LSTM으로 문장을 전달하고. 로그 소프트 맥스를 씌워서 argmax를 


In [9]:
def prepare_sequence(seq, to_ix):
  idxs = [to_ix[w] for w in seq]
  return torch.tensor(idxs, dtype=torch.long)

training_data = [
  ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
  ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_idx = {}
for sent, tags in training_data:
  for word in sent:
    if word not in word_to_idx:
      word_to_idx[word] = len(word_to_idx)

print(word_to_idx)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

EMBEDDING_DIM = 6
HIDDEM_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [0]:
class LSTMTagger(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
    super(LSTMTagger, self).__init__()
    self.hidden_dim = hidden_dim

    self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

    # LSTM은 워드 임베딩을 입력 받고 히든 스테이트를 출력한다.
    # hidden_dim 으로 정의한 차원으로
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)

    # 리니어 레이어는 히든 스테이트 공간에서 태그 스페이스로 매핑한다.
    self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

  def forward(self, sentence):
    embeds = self.word_embeddings(sentence)
    print(embeds)
    print(embeds.shape)
    lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
    print(lstm_out)
    print(_)
    tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
    tag_scores = F.log_softmax(tag_space, dim=1)
    return tag_scores

In [18]:
model = LSTMTagger(embedding_dim=EMBEDDING_DIM,
                   hidden_dim=HIDDEM_DIM,
                   vocab_size=len(word_to_idx),
                   tagset_size=len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
print(model)

LSTMTagger(
  (word_embeddings): Embedding(9, 6)
  (lstm): LSTM(6, 6)
  (hidden2tag): Linear(in_features=6, out_features=3, bias=True)
)


In [19]:
with torch.no_grad():
  print(training_data[0][0])
  inputs = prepare_sequence(training_data[0][0], word_to_idx)
  print(inputs)
  tag_scores = model(inputs)
  print(tag_scores)

['The', 'dog', 'ate', 'the', 'apple']
tensor([0, 1, 2, 3, 4])
tensor([[-0.5454,  0.0985, -0.1218,  0.3274, -1.0497,  0.2502],
        [ 1.7113,  0.9126, -0.0169,  1.4172, -0.4237, -0.0769],
        [ 0.5362,  0.6485,  0.1329, -0.1203,  1.4280,  0.5847],
        [-1.3204, -0.2781, -1.4720,  1.2683,  0.7452, -0.3816],
        [-0.6646, -1.4525,  0.1274,  1.6439, -0.3111, -1.0572]])
torch.Size([5, 6])
tensor([[[-6.8528e-02, -4.2312e-05,  1.3350e-01,  1.4006e-01, -6.3120e-03,
           1.0050e-01]],

        [[-1.0207e-01,  6.3780e-02,  1.0711e-01, -1.4232e-01, -4.9737e-02,
           1.7420e-01]],

        [[-8.0842e-02,  5.6233e-03,  1.2488e-01, -2.1802e-01, -5.5637e-02,
          -1.1259e-01]],

        [[-1.9890e-01,  1.3407e-01,  2.6237e-01,  1.1725e-01,  7.7432e-02,
           4.1564e-02]],

        [[-3.0839e-01,  1.5909e-01,  2.2927e-01,  1.4703e-01,  5.4129e-02,
           5.1054e-02]]])
(tensor([[[-0.3084,  0.1591,  0.2293,  0.1470,  0.0541,  0.0511]]]), tensor([[[-0.6432,  0.50

In [24]:
for epoch in range(300):
  for sentence, tags in training_data:
    model.zero_grad()

    sentence_in = prepare_sequence(sentence, word_to_idx)
    targets = prepare_sequence(tags, tag_to_ix)

    print(sentence)
    print(tags)
    tag_scores = model(sentence_in)

    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer.step()

    break
  break

['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
tensor([[-0.9762,  0.2633, -0.6479, -0.3214, -0.4275, -0.2357],
        [ 2.1305,  0.7399,  0.1118,  2.0102, -0.6656, -0.1144],
        [ 0.6298,  1.1102,  0.1137,  0.2235,  1.9730,  0.9110],
        [-1.6952, -0.2764, -1.7232,  0.9574,  0.8487, -0.5334],
        [-0.1292, -1.6864,  0.5710,  2.2424, -0.2707, -1.0105]],
       grad_fn=<EmbeddingBackward>)
torch.Size([5, 6])
tensor([[[-0.6522, -0.0516, -0.1285,  0.5599, -0.0093,  0.0445]],

        [[-0.0136,  0.6433,  0.4422, -0.0764, -0.1803,  0.7546]],

        [[-0.6472, -0.2401, -0.3885, -0.8307,  0.3115,  0.8788]],

        [[-0.8867,  0.0977, -0.3808,  0.6480,  0.1277,  0.0385]],

        [[-0.0270,  0.7632,  0.5838,  0.2779,  0.0957,  0.2289]]],
       grad_fn=<StackBackward>)
(tensor([[[-0.0270,  0.7632,  0.5838,  0.2779,  0.0957,  0.2289]]],
       grad_fn=<StackBackward>), tensor([[[-0.4805,  1.0634,  0.7367,  0.7821,  0.3151,  1.1021]]],
       grad_fn=<Sta

In [26]:
with torch.no_grad():
  inputs = prepare_sequence(training_data[0][0], word_to_idx)
  tag_scores = model(inputs)

  # 문장은 "the dog ate the apple". 
  # word 인 i에 대해서 예측된 태그는 최대 점수인 태그다.
  # 여기서 아래 시퀀스에서 예측된 시퀀스는 0 1 2 0 1
  # 0이 row 1에서의 최댓값의 인덱스이기 때문에
  # 1이 row 2에서의 최댓값의 인덱스이기 때문에
  # DET NOUN VERB DET NOUN 인 정답 시퀀스
  print(tag_scores)

tensor([[-0.9764,  0.2633, -0.6481, -0.3215, -0.4274, -0.2358],
        [ 2.1306,  0.7398,  0.1118,  2.0104, -0.6656, -0.1144],
        [ 0.6299,  1.1103,  0.1137,  0.2236,  1.9732,  0.9111],
        [-1.6953, -0.2764, -1.7232,  0.9574,  0.8487, -0.5334],
        [-0.1289, -1.6864,  0.5712,  2.2426, -0.2706, -1.0105]])
torch.Size([5, 6])
tensor([[[-0.6523, -0.0516, -0.1287,  0.5600, -0.0093,  0.0445]],

        [[-0.0136,  0.6433,  0.4423, -0.0764, -0.1803,  0.7547]],

        [[-0.6472, -0.2402, -0.3886, -0.8307,  0.3116,  0.8788]],

        [[-0.8868,  0.0978, -0.3810,  0.6480,  0.1276,  0.0385]],

        [[-0.0270,  0.7633,  0.5839,  0.2777,  0.0957,  0.2292]]])
(tensor([[[-0.0270,  0.7633,  0.5839,  0.2777,  0.0957,  0.2292]]]), tensor([[[-0.4804,  1.0635,  0.7367,  0.7821,  0.3151,  1.1025]]]))
tensor([[-0.0235, -4.1235, -4.9611],
        [-5.3857, -0.0140, -4.6774],
        [-4.3891, -5.6104, -0.0162],
        [-0.0098, -5.2722, -5.3769],
        [-4.3830, -0.0140, -6.5810]])
